In [1]:
#import the needed libraries
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
#create the SparkSession
spark = (SparkSession
        .builder
        .appName("Spark Ejs")
        .getOrCreate())

## Exercises from Chapter 2

In [ ]:
#a

#route of the file to read
route = "Desktop\Big Data\el_quijote.txt"

#loads the file into data
#Spark supports csv, json and text files(txt) 
data = (spark.read.format("text")
        .option("header", "false")
        .load(route))
    
data.count() #2186
#data.show()  #shows the top 20 rows in tabular form
#data.show(5) #shows the selected number of rows in tabular form
#data.show(truncate = False) #shows the whole text (doesnt put "...", shows all)
#data.show(5, truncate = False) #shows the selected number of rows
#data.show(5, truncate = False, vertical = False) #shows the selected number of rows, truncated to 25 character in horizontal
#in Python is allowed to put False when expected int because I think it internally cast it to int (False is 0)

#data.head()  #returns the first row
#data.first() #returns the first row
#data.head(5) #takes the number selected of rows from the start and puts them in an array 
#data.take(5) #takes the number selected of rows from the start and puts them in an array (works as head(<n>) does) 
#data.tail(5) #takes the number selected of rows from the end and puts them in an array

#There is no diference in the behaviour between head and first, the only difference is that head can take parametres and first can't
#head(<n>) and take(<n>) return the same thing



In [31]:
#b

route = "Desktop/Big Data/DataSets Libro Spark/mnm_dataset.csv.txt"

data = spark.read.format("csv") \
    .option("header", "true") \
    .load(route)

# totalByState = (data.select("State", "Color", "Count")
#     .groupBy("State", "Color")
#     .agg(sum("Count").alias("Total")))

# totalByState.show()

# totalByState.select("State", "Color", "Total") \
#     .groupBy("State") \
#     .agg(max("Total").alias("Max")) \
#     .orderBy(desc("State")).show()


# caCount = (data.select("State", "Color", "Count")
#     .where(col("State") == "CA")
#     .groupBy("State", "Color")
#     .agg(sum("Count").alias("Total"))
#     .orderBy(desc("Total")).show())

# data.select("State", "Color", "Count") \
#     .where((col("State") == "NV") | (col("State") == "TX") | (col("State") == "CA") | (col("State") == "CO")) \
#     .groupBy("State", "Color") \
#     .agg(sum("Count").alias("Total")) \
#     .orderBy(asc("State"),asc("Color")) \
#     .show()

data.select("State", "Color", "Count") \
    .groupBy("State", "Color") \
    .agg(max("Count").alias("Max"), avg("Count").alias("Average")) \
    .orderBy(asc("State"), asc("Color")) \
    .show()
    

+-----+------+---+------------------+
|State| Color|Max|           Average|
+-----+------+---+------------------+
|   AZ|  Blue| 99| 54.99449877750611|
|   AZ| Brown| 99|54.350412249705535|
|   AZ| Green| 99| 54.82219570405728|
|   AZ|Orange| 99| 54.28300769686205|
|   AZ|   Red| 99|54.637135922330096|
|   AZ|Yellow| 99| 54.98548972188634|
|   CA|  Blue| 99| 55.59762944479102|
|   CA| Brown| 99|55.740395809080326|
|   CA| Green| 99|54.268717353453276|
|   CA|Orange| 99|54.502715751357876|
|   CA|   Red| 99| 55.26992753623188|
|   CA|Yellow| 99|  55.8693967902601|
|   CO|  Blue| 99| 55.11032448377581|
|   CO| Brown| 99| 56.57729468599034|
|   CO| Green| 99| 54.71336835960304|
|   CO|Orange| 99|55.402557856272836|
|   CO|   Red| 99|55.089285714285715|
|   CO|Yellow| 99| 55.22254503195816|
|   NM|  Blue| 99| 55.03663003663004|
|   NM| Brown| 99|55.392412566686424|
+-----+------+---+------------------+
only showing top 20 rows



## Exercises from Chapter 3


a)

------Blog DataSet exercises------

In [3]:
#Load the Data from the example of the book
route = "Desktop/Big Data/DataSets Libro Spark/blogs.json"

In [4]:
#Sets the schema with DDl
schema = "`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"

#Creates a DataFrame with the data from the file and the schema defined with DDL
blogs_df = spark.read.schema(schema).json(route)

In [5]:
#Sets the schema with the DataFrame API
schema = StructType([StructField('Id', IntegerType(), False),
                        StructField('First', StringType(), False),
                        StructField('Last', StringType(), False),
                        StructField('URL', StringType(), False), 
                        StructField ('Published', DateType(), False),
                        StructField('Hits', IntegerType(), False),
                        StructField('Campaigns', ArrayType(StringType()), False)])
#The third parameter indicates if that field is nullable or not


#Creates a DataFrame with the data from the file and the schema defined with the DataFrame API
blogs_df = spark.read.schema(schema).json(route)

In [6]:
#Show the Data
blogs_df.show()

#Show the Schema 
print(blogs_df.printSchema())

#Print the Schema
print(blogs_df.schema)

+---+---------+-------+----+---------+-----+--------------------+
| Id|    First|   Last| URL|Published| Hits|           Campaigns|
+---+---------+-------+----+---------+-----+--------------------+
|  1|    Jules|  Damji|null|     null| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|null|     null| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|null|     null| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|null|     null|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|null|     null|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|null|     null|25568| [twitter, LinkedIn]|
+---+---------+-------+----+---------+-----+--------------------+

root
 |-- Id: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Published: date (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)

None
StructType(List(Struc

In [7]:
#Add a column called Big Hitters for people which hits are over 10000
(blogs_df.withColumn("Big Hitters", expr("Hits > 10000")).show())

#Concatenate First, Last and Id to create AuthorsId

#(blogs_df.withColumn("AuthorsId", expr("First || Last  || Id")).show())
(blogs_df.withColumn("AuthorsId", concat(col("First"), lit("_"), col("Last"), lit("-"), col("Id"))).show())

+---+---------+-------+----+---------+-----+--------------------+-----------+
| Id|    First|   Last| URL|Published| Hits|           Campaigns|Big Hitters|
+---+---------+-------+----+---------+-----+--------------------+-----------+
|  1|    Jules|  Damji|null|     null| 4535| [twitter, LinkedIn]|      false|
|  2|   Brooke|  Wenig|null|     null| 8908| [twitter, LinkedIn]|      false|
|  3|    Denny|    Lee|null|     null| 7659|[web, twitter, FB...|      false|
|  4|Tathagata|    Das|null|     null|10568|       [twitter, FB]|       true|
|  5|    Matei|Zaharia|null|     null|40578|[web, twitter, FB...|       true|
|  6|  Reynold|    Xin|null|     null|25568| [twitter, LinkedIn]|       true|
+---+---------+-------+----+---------+-----+--------------------+-----------+

+---+---------+-------+----+---------+-----+--------------------+---------------+
| Id|    First|   Last| URL|Published| Hits|           Campaigns|      AuthorsId|
+---+---------+-------+----+---------+-----+-----------

------San Francisco Fire Department Exercises-------

In [8]:
#Set the schema
schema = StructType([StructField("CallNumber", IntegerType(), True),
                              StructField("UnitID", StringType(), True),
                              StructField("IncidentNumber", IntegerType(), True),
                              StructField("CallType", StringType(), True), 
                              StructField("CallDate", StringType(), True), 
                              StructField("WatchDate", StringType(), True),
                              StructField("CallFinalDisposition", StringType(), True),
                              StructField("AvailableDtTm", StringType(), True),
                              StructField("Address", StringType(), True), 
                              StructField("City", StringType(), True), 
                              StructField("Zipcode", IntegerType(), True), 
                              StructField("Battalion", StringType(), True), 
                              StructField("StationArea", StringType(), True), 
                              StructField("Box", StringType(), True), 
                              StructField("OriginalPriority", StringType(), True), 
                              StructField("Priority", StringType(), True), 
                              StructField("FinalPriority", IntegerType(), True), 
                              StructField("ALSUnit", BooleanType(), True), 
                              StructField("CallTypeGroup", StringType(), True),
                              StructField("NumAlarms", IntegerType(), True),
                              StructField("UnitType", StringType(), True),
                              StructField("UnitSequenceInCallDispatch", IntegerType(), True),
                              StructField("FirePreventionDistrict", StringType(), True),
                              StructField("SupervisorDistrict", StringType(), True),
                              StructField("Neighborhood", StringType(), True),
                              StructField("Location", StringType(), True),
                              StructField("RowID", StringType(), True),
                              StructField("Delay", FloatType(), True)])
#Load the data from the SF Fire Department
route = "Desktop/Big Data/DataSets Libro Spark/Fire_Incidents.csv"

data = spark.read.format("csv").option("header", "True").schema(schema).load(route)


In [9]:
#Get the Incidents that are not Medical Incidents
(data.where(col("CallType") != "Medical Incidents").show(5, 25, True))

-RECORD 0-----------------------------------------------
 CallNumber                 | 20110016                  
 UnitID                     | T13                       
 IncidentNumber             | 2003235                   
 CallType                   | Structure Fire            
 CallDate                   | 01/11/2002                
 WatchDate                  | 01/10/2002                
 CallFinalDisposition       | Other                     
 AvailableDtTm              | 01/11/2002 01:51:44 AM    
 Address                    | 2000 Block of CALIFORN... 
 City                       | SF                        
 Zipcode                    | 94109                     
 Battalion                  | B04                       
 StationArea                | 38                        
 Box                        | 3362                      
 OriginalPriority           | 3                         
 Priority                   | 3                         
 FinalPriority              | 3

In [10]:
#How many distinct CallTypes were recorded as the causes of the fire calls?
print(data.select("CallType").distinct().count())

30


In [11]:
#Rename the Column Delay to ResponseDelayedInMins and get the ones above 5
(data.withColumnRenamed("Delay", "ResponsableDelayedInMins").show(5, 25, True))

-RECORD 0-----------------------------------------------
 CallNumber                 | 20110016                  
 UnitID                     | T13                       
 IncidentNumber             | 2003235                   
 CallType                   | Structure Fire            
 CallDate                   | 01/11/2002                
 WatchDate                  | 01/10/2002                
 CallFinalDisposition       | Other                     
 AvailableDtTm              | 01/11/2002 01:51:44 AM    
 Address                    | 2000 Block of CALIFORN... 
 City                       | SF                        
 Zipcode                    | 94109                     
 Battalion                  | B04                       
 StationArea                | 38                        
 Box                        | 3362                      
 OriginalPriority           | 3                         
 Priority                   | 3                         
 FinalPriority              | 3

In [12]:
#Convert the columns CallDate, WatchDate, AvailableDtTm and name them IncidentDate, OnWatchDate, AvailableDtTS
dataRenamed = (data.withColumnRenamed("CallDate", "IncidentDate").withColumn("IncidentDate", to_date(col("IncidentDate"), "MM/dd/yyyy")) \
               .withColumnRenamed("WatchDate", "OnWatchDate").withColumn("OnWatchDate", to_date(col("OnWatchDate"), "MM/dd/yyyy")) \
               .withColumnRenamed("AvailableDtTm", "AvailableDtTS").withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTS"), "MM/dd/yyyy hh:mm:ss a")))
dataRenamed.show(5, 25, True)

-RECORD 0-----------------------------------------------
 CallNumber                 | 20110016                  
 UnitID                     | T13                       
 IncidentNumber             | 2003235                   
 CallType                   | Structure Fire            
 IncidentDate               | 2002-01-11                
 OnWatchDate                | 2002-01-10                
 CallFinalDisposition       | Other                     
 AvailableDtTS              | 2002-01-11 01:51:44       
 Address                    | 2000 Block of CALIFORN... 
 City                       | SF                        
 Zipcode                    | 94109                     
 Battalion                  | B04                       
 StationArea                | 38                        
 Box                        | 3362                      
 OriginalPriority           | 3                         
 Priority                   | 3                         
 FinalPriority              | 3

In [15]:
#Obtain the years in which an incident has happened (years must only show once)
(dataRenamed.select(year(col("IncidentDate"))).distinct().orderBy(col("IncidentDate").asc()).show())

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



In [18]:
#What were the most common types of fire calls?? (Order the calls by their popularity)
(dataRenamed.groupBy("CallType") \
            .agg(sum(col("NumAlarms")).alias("Number of calls")) \
            .orderBy(col("Number of calls").desc()) \
            .show())

+--------------------+---------------+
|            CallType|Number of calls|
+--------------------+---------------+
|    Medical Incident|         113794|
|      Structure Fire|          24193|
|              Alarms|          19406|
|   Traffic Collision|           7013|
|Citizen Assist / ...|           2524|
|               Other|           2166|
|        Outside Fire|           2094|
|        Vehicle Fire|            854|
|Gas Leak (Natural...|            764|
|        Water Rescue|            755|
|Odor (Strange / U...|            490|
|   Electrical Hazard|            482|
|Elevator / Escala...|            453|
|Smoke Investigati...|            391|
|          Fuel Spill|            193|
|              HazMat|            124|
|Industrial Accidents|             94|
|           Explosion|             89|
|Train / Rail Inci...|             57|
|  Aircraft Emergency|             36|
+--------------------+---------------+
only showing top 20 rows



In [21]:
#Compute the sum of alarms, the average response time, and the minimum and maximum response times to all fire calls
(dataRenamed.select(sum(col("NumAlarms")).alias("Number of calls"), avg(col("Delay")).alias("Average response time"), \
                  min(col("Delay")).alias("Minimum response time"), max(col("Delay")).alias("Maximum response time")) \
            .show())

+---------------+---------------------+---------------------+---------------------+
|Number of calls|Average response time|Minimum response time|Maximum response time|
+---------------+---------------------+---------------------+---------------------+
|         176170|    3.892364154521585|          0.016666668|              1844.55|
+---------------+---------------------+---------------------+---------------------+



In [23]:
#What were all the different types of fire calls in 2018?
(dataRenamed.where(year(col("IncidentDate")) == 2018).select("CallType").distinct().show())

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|           Explosion|
|        Vehicle Fire|
|  Suspicious Package|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
|        Water Rescue|
|   Electrical Hazard|
|      Structure Fire|
|    Medical Incident|
|          Fuel Spill|
|Smoke Investigati...|
|Train / Rail Inci...|
+--------------------+



In [30]:
#What months within the year 2018 saw the highest number of fire calls?
(dataRenamed.where(year(col("IncidentDate")) == 2018) \
            .withColumn("Month of call", month(col("IncidentDate"))) \
            .groupBy("Month of call", "CallType") \
            .agg(sum("NumAlarms").alias("Number of alarms")) \
            .orderBy(col("Number of alarms").desc()) \
            .show())

+-------------+----------------+----------------+
|Month of call|        CallType|Number of alarms|
+-------------+----------------+----------------+
|            5|Medical Incident|             769|
|           10|Medical Incident|             725|
|            8|Medical Incident|             698|
|            3|Medical Incident|             695|
|            1|Medical Incident|             692|
|            6|Medical Incident|             690|
|            7|Medical Incident|             665|
|            9|Medical Incident|             654|
|            4|Medical Incident|             646|
|            2|Medical Incident|             635|
|           11|Medical Incident|             135|
|            3|          Alarms|             133|
|            8|          Alarms|             131|
|           10|          Alarms|             130|
|            1|          Alarms|             122|
|            7|          Alarms|             117|
|            5|          Alarms|             107|


In [32]:
#Which neighborhood in San Francisco generated the most fire calls in 2018?
(dataRenamed.where(year(col("IncidentDate")) == 2018) \
            .groupBy("Neighborhood") \
            .agg(sum("NumAlarms").alias("Number of alarms")) \
            .orderBy(col("Number of alarms").desc()) \
            .show())

+--------------------+----------------+
|        Neighborhood|Number of alarms|
+--------------------+----------------+
|          Tenderloin|            1393|
|     South of Market|            1053|
|             Mission|             913|
|Financial Distric...|             774|
|Bayview Hunters P...|             522|
|    Western Addition|             352|
|     Sunset/Parkside|             346|
|            Nob Hill|             296|
|        Hayes Valley|             291|
|      Outer Richmond|             262|
| Castro/Upper Market|             252|
|         North Beach|             231|
|           Excelsior|             212|
|        Potrero Hill|             210|
|  West of Twin Peaks|             210|
|           Chinatown|             194|
|              Marina|             191|
|     Pacific Heights|             191|
|         Mission Bay|             178|
|      Bernal Heights|             171|
+--------------------+----------------+
only showing top 20 rows



In [33]:
#Which neighborhoods had the worst response times to fire calls in 2018?
(dataRenamed.where(year(col("IncidentDate")) == 2018) \
            .groupBy("Neighborhood") \
            .agg(max("Delay").alias("Response time")) \
            .orderBy(col("Response time").desc()) \
            .show())

+--------------------+-------------+
|        Neighborhood|Response time|
+--------------------+-------------+
|           Chinatown|    491.26666|
|Financial Distric...|    406.63333|
|          Tenderloin|    340.48334|
|      Haight Ashbury|    175.86667|
|Bayview Hunters P...|        155.8|
|     Pacific Heights|    129.01666|
|        Potrero Hill|        109.8|
|        Inner Sunset|    106.13333|
|     South of Market|     94.71667|
|      Inner Richmond|    90.433334|
|           Excelsior|     83.76667|
| Castro/Upper Market|     74.13333|
|    Western Addition|    67.916664|
|            Nob Hill|        67.45|
|             Mission|    54.666668|
|    Presidio Heights|    52.883335|
|       Outer Mission|    43.383335|
|  West of Twin Peaks|         43.2|
|         North Beach|    40.933334|
|            Presidio|    38.816666|
+--------------------+-------------+
only showing top 20 rows



In [34]:
#Which week in the year in 2018 had the most fire calls?
(dataRenamed.where(year(col("IncidentDate")) == 2018) \
            .groupBy(weekofyear(col("IncidentDate"))) \
            .agg(sum("NumAlarms").alias("Number of alarms")) \
            .orderBy(col("Number of alarms").desc()) \
            .show())

+------------------------+----------------+
|weekofyear(IncidentDate)|Number of alarms|
+------------------------+----------------+
|                      22|             259|
|                      40|             255|
|                      43|             252|
|                      25|             250|
|                       1|             246|
|                      44|             244|
|                      13|             243|
|                      11|             243|
|                      32|             243|
|                       5|             236|
|                      18|             236|
|                      23|             235|
|                      42|             234|
|                       2|             234|
|                      31|             234|
|                       8|             233|
|                      19|             233|
|                      16|             232|
|                      34|             232|
|                      10|      

In [ ]:
//Is there a correlation between neighborhood, zip code, and number of fire calls?
dataRenamed.groupBy("Neighborhood", "Zipcode")
           .agg(sum("NumAlarms").alias("Number of alarms"))
           .orderBy(desc("Zipcode"), desc("Number of alarms"))
           .select("Neighborhood", "Zipcode", "Number of alarms").show(50)

//Each neighborhood has a series of zipcodes assigned. For example: The Neighborhood Mission has 94114, 94110 and 94103
//But some neighborhoods have the same zipcode as others so it not a way to identify them
//I haven't found a correlation between them


In [52]:
#Is there a correlation between neighborhood, zip code, and number of fire calls?
(dataRenamed.groupBy("Neighborhood", "Zipcode") \
            .agg(sum("NumAlarms").alias("Number of alarms")) \
            .orderBy(col("Zipcode").desc(), col("Number of alarms").desc()) \
            .show())

+--------------------+-------+----------------+
|        Neighborhood|Zipcode|Number of alarms|
+--------------------+-------+----------------+
|         Mission Bay|  94158|             842|
|        Potrero Hill|  94158|              38|
|                None|  94158|              11|
|   Visitacion Valley|  94134|            2442|
|             Portola|  94134|            1688|
|           Excelsior|  94134|             407|
|Bayview Hunters P...|  94134|             292|
|        McLaren Park|  94134|             171|
|                None|  94134|              29|
|         North Beach|  94133|            3711|
|           Chinatown|  94133|            1874|
|        Russian Hill|  94133|             448|
|            Nob Hill|  94133|             238|
|           Lakeshore|  94132|            2663|
|Oceanview/Merced/...|  94132|             846|
|     Sunset/Parkside|  94132|             555|
|  West of Twin Peaks|  94132|             262|
|          Twin Peaks|  94131|          

In [53]:
#How can we use Parquet files or SQL tables to store this data and read it back?
dataRenamed.write.format("parquet").save("Desktop/Big Data/DataSets Cacheados/dataRenamed2.parquet")
#dataRenamed.write.format("parquet").saveAsTable("<DB>.<Table>")
